<a href="https://colab.research.google.com/github/cinnamonmeal0/books_scraping/blob/main/kw%E6%B4%97%E3%81%84%E5%87%BA%E3%81%97%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%97%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 準備
---
#### まず初めにこのセルを実行してください。実行完了してからでないとエラーが出てしまいます。


In [ ]:
# 必要なライブラリなどインストール
! pip install beautifulsoup4
! pip install selenium
! pip install google-colab-selenium

In [ ]:
duplication_set = set()

In [ ]:
def find_last_parenthesis_index(text):
  """
  文字列の最後尾が)または）であれば、(または（が前から何番目かを取得する関数

  Args:
    text: 処理対象の文字列

  Returns:
    (または（が前から何番目か、または-1 (最後尾に)または）がない場合)
  """

  # 最後尾が)か）かをチェック
  if text.endswith(")") or text.endswith("）"):
    # 最後尾が)の場合
    if text.endswith(")"):
      return text.rfind("(")
    # 最後尾が）の場合
    else:
      return text.rfind("（")
  else:
    return -1

def check_duplicate(text):
  # 正規表現で【】とその中のテキストを削除
  text = re.sub(r'【.+?】', '', text)
  text = re.sub(r'《.+?》', '', text)
  text = re.sub(r'［.+?］', '', text)
  text = re.sub(r'●.+?●', '', text)
  text = re.sub(r'＜.+?＞', '', text)
  # 最後尾に()があるかチェックし、あれば削除
  index = find_last_parenthesis_index(text)
  if index != -1:
    text = text[:index]
  return text

def delete_unnecessary_words(book_title):
  # 不要ワードはここに追加する
  unnecessary_words = ["分冊版", "フルカラー", "単話", "THE COMIC", "上", "下", "vol.1", "act.1", "連載版"]

  for word in unnecessary_words:
    if word in book_title:
      print(book_title)
      book_title = book_title.replace(word, "")

  # スペースが重複している場合は一つにまとめる
  book_title = " ".join(book_title.split())

  return book_title


### これより下記のセルは1つずつ順番に実行してください。処理が終わるまで次のセルを実行しないでください。
何かエラーが起きて実行できないときは、Slack(@Nozomi Sugaya)まで、エラーの内容(コピペで大丈夫です)含めて連絡お願いします。

### コミックシーモア 上位20位取得
---


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

cmoa_data_to_insert = []

get_dict = {"少女マンガ":"genre/?id=20", "女性マンガ":"genre/?id=2", "少年マンガ":"genre/?id=12", "青年マンガ":"genre/?id=13", "BLコミック":"genre/?id=24", "TLコミック":"genre/?id=34","シーモア先行":"precede", "オリジナル":"original", "オトナコミック":"genre/?id=11"}
# スクレイピング対象の URL にリクエストを送り HTML を取得する
for key,v in get_dict.items():
  res = requests.get('https://www.cmoa.jp/search/purpose/ranking/' + v)

  # レスポンスの HTML から BeautifulSoup オブジェクトを作る
  soup = BeautifulSoup(res.text, 'html.parser')
  print("------")
  print("【"+ key + "】")
  for k in range(20):
      elems = soup.select(f'#ranking_result_list > li:nth-of-type({k + 1}) > div.search_result_box_right > div.search_result_box_right_sec1 > p > a')
      if elems:
          link = elems[0]['href']
          full_path = "https://www.cmoa.jp" + link
          text = elems[0].text.strip()
          if text:
              text = check_duplicate(text)
              text = delete_unnecessary_words(text)
              if text[:5] in [item[:5] for item in duplication_set] and text[-3:] in [item[-3:] for item in duplication_set]:
                continue
              print(text, full_path)
              cmoa_data_to_insert.append([text, full_path,"コミックシーモア上位20位" + "【"+ key + "】"])
              duplication_set.add(text)
              # print(text)
st.append_rows(values=cmoa_data_to_insert)

------
【少女マンガ】
傷モノの花嫁～虐げられた私が、皇國の鬼神に見初められた理由～ 分冊版
------
【女性マンガ】
悪役令嬢レベル99 ～私は裏ボスですが魔王ではありません～ https://www.cmoa.jp/title/209389/
元夫から「ロミオメール」が届いた件について https://www.cmoa.jp/title/263136/
ヤンデレ魔法使いは石像の乙女しか愛せない 魔女は愛弟子の熱い口づけでとける https://www.cmoa.jp/title/236751/
純潔の罪 https://www.cmoa.jp/title/184272/
今度は絶対に邪魔しませんっ! https://www.cmoa.jp/title/170828/
BLゲームの主人公の弟であることに気がつきました https://www.cmoa.jp/title/234498/
精霊魔法が使えない無能だと婚約破棄されたので、義妹の奴隷になるより追放を選びました https://www.cmoa.jp/title/267617/
破滅の聖女は運命の夫の溺愛から逃れたい https://www.cmoa.jp/title/285946/
名も知らず https://www.cmoa.jp/title/286269/
推定悪役令嬢は国一番のブサイクに嫁がされるようです https://www.cmoa.jp/title/272284/
悦覧禁止-暁月教授の怪異検証- https://www.cmoa.jp/title/284271/
瓜を破る https://www.cmoa.jp/title/217954/
結界師の一輪華 https://www.cmoa.jp/title/251291/
甘いお菓子の後は甘い溺愛を～婚約破棄された令嬢は辺境伯子息に溺愛される～ https://www.cmoa.jp/title/286651/
にぶんのいち夫婦 https://www.cmoa.jp/title/171657/
悪女(と誤解される私)が腹黒王太子様の愛され妃になりそうです!? https://www.cmoa.jp/title/249073/
木更津くんの××が見たい https://www.cmoa.jp/title/207107/
未満。だけど愛

{'spreadsheetId': '1RiLchx4pkQ_vdngW3UIG49xuhoZEThM4rAhuTPJQdH4',
 'tableRange': "'スクレイピング結果'!A1:C21",
 'updates': {'spreadsheetId': '1RiLchx4pkQ_vdngW3UIG49xuhoZEThM4rAhuTPJQdH4',
  'updatedRange': "'スクレイピング結果'!A22:C156",
  'updatedRows': 135,
  'updatedColumns': 3,
  'updatedCells': 405}}

### コミックシーモア発売予定作品
---


In [ ]:
import requests
import re
from bs4 import BeautifulSoup
cmoa_schedule_data_to_insert = []
# スクレイピング対象の URL にリクエストを送り HTML を取得する
res = requests.get('https://www.cmoa.jp/newrelease/schedule/boy/')

def get_release_books(number):
  soup = BeautifulSoup(res.content, 'html.parser', from_encoding='utf-8')
  title = soup.select_one(f'#home > section.co_container.pc_with > div > div > section.newrelease_area > div:nth-of-type({number}) > div')
  try:
    text_content = title.text.strip()
    match = re.search(r"\((\d+)\)", text_content)

    # 抽出結果を取得
    if match:
        # 正規表現で括弧内の数字を抽出
        count = int(match.group(1))

    print("------")
    print("【"+text_content+ "】")

    for k in range(count+1):
      elems = soup.select(f'#home > section.co_container.pc_with > div > div > section.newrelease_area > div:nth-of-type({number}) > ul > li:nth-of-type({k +1}) > div > a')
      role = soup.select_one(f'#home > section.co_container.pc_with > div > div > section.newrelease_area > div:nth-of-type({number}) > ul > li:nth-of-type({k +1}) > div > div.vol_num')
      if role != None:
        role_text = role.text
        if role_text == "1巻" or role_text.startswith("1-"):
          if elems:
            link = elems[0]['href']
            full_path = "https://www.cmoa.jp" + link
            text = elems[0].text.strip()
            if text:
                text = check_duplicate(text)
                text = delete_unnecessary_words(text)
                if text[:5] in [item[:5] for item in duplication_set] and text[-3:] in [item[-3:] for item in duplication_set]:
                  continue
                print(text, full_path)
                cmoa_schedule_data_to_insert.append([text, full_path, "コミックシーモア発売予定" + "【"+ text_content + "】"])
                duplication_set.add(text)

  except AttributeError:
    print("Error: 'title' is NoneType. Exiting the program.")
    exit()

# tommorrow
get_release_books(1)

# the day after tomorrow ..
get_release_books(2)
get_release_books(3)

st.append_rows(values=cmoa_schedule_data_to_insert)


------
【3/5(火)発売予定作品 (22)】
------
【3/6(水)発売予定作品 (58)】
------
【3/7(木)発売予定作品 (39)】


{'spreadsheetId': '1RiLchx4pkQ_vdngW3UIG49xuhoZEThM4rAhuTPJQdH4',
 'tableRange': "'スクレイピング結果'!A1:C168",
 'updates': {'spreadsheetId': '1RiLchx4pkQ_vdngW3UIG49xuhoZEThM4rAhuTPJQdH4',
  'updatedRange': "'スクレイピング結果'!A169"}}

### Renta! 1位～5位取得
---


In [ ]:
import google_colab_selenium as gs
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


options = Options()
options.add_argument("--headless")

get_list = ["少女漫画","少年漫画","青年漫画","映像化作品","ボーイズラブ漫画","ティーンズラブコミック","レディースコミック","ヤングレディース","美女・美少女","ルポ・エッセイ","タテコミ"]
renta_ranking_data_to_insert = []

print("connectiong to remote browser...")
driver = gs.Chrome(options=options)

driver.get("https://renta.papy.co.jp/renta/sc/frm/page/ranking_c.htm?type=week")

rankign_item_wrap_list = driver.find_elements(
    By.XPATH, '//div[@id="ranking_item"]/div[@class="itemRank_wrap"]/h2/span'
)
for ranking_item_wrap in rankign_item_wrap_list:
    if ranking_item_wrap.text not in get_list:
      continue
    print("------")
    print(f"【{ranking_item_wrap.text}】")
    item_contents_list = driver.find_elements(
        By.XPATH,
        f'//div[@id="ranking_item"]/div[@id="{ranking_item_wrap.text}"]/div[@class="contentsRank_wrap"]/div[@class="itemContents"]',
    )
    count = 0
    for item_contents in item_contents_list:
        manga_title = item_contents.find_element(
            By.XPATH,
            "./a/p",
        ).text
        manga_url = item_contents.find_element(
            By.XPATH,
            "./a",
        ).get_attribute("href")
        # 最初と最後の数文字で検出するように変更
        manga_title = check_duplicate(manga_title)
        manga_title = delete_unnecessary_words(manga_title)
        if manga_title[:5] in [item[:5] for item in duplication_set] and manga_title[-3:] in [item[-3:] for item in duplication_set]:
          continue
        print(f"{manga_title} {manga_url}")
        renta_ranking_data_to_insert.append([manga_title, manga_url, "renta上位" + "【"+ ranking_item_wrap.text + "】"])
        duplication_set.add(manga_title)
        count += 1
        if count == 5:
          break

st.append_rows(values=renta_ranking_data_to_insert)
driver.quit()

connectiong to remote browser...


<IPython.core.display.Javascript object>

------
【少女漫画】
------
【少年漫画】
------
【青年漫画】
------
【映像化作品】
------
【ボーイズラブ漫画】
------
【ティーンズラブコミック】
成瀬くんは溺愛オオカミ―理想の彼氏は幼なじみでした― https://renta.papy.co.jp/renta/sc/frm/item/164095/title/892901/
------
【レディースコミック】
------
【ヤングレディース】
サレ妻の復讐～魔性の刺青～ https://renta.papy.co.jp/renta/sc/frm/item/226756/title/2492400/
ビジネス婚─好きになったら離婚します─ https://renta.papy.co.jp/renta/sc/frm/item/342699/title/2435722/
------
【美女・美少女】
レス妻が性感エステで「また…イク…っ」～うねる指で奥までほぐされ… https://renta.papy.co.jp/renta/sc/frm/item/188855/title/2516380/
幼馴染にイかされるなんて…！同居初日に喧嘩エッチ https://renta.papy.co.jp/renta/sc/frm/item/143887/title/2522457/
家族だけど大丈夫 https://renta.papy.co.jp/renta/sc/frm/item/311723/title/2522441/
------
【タテコミ】
奉公物語～僕のお嬢様～ https://renta.papy.co.jp/renta/sc/frm/item/342470/title/2509889/
旦那の浮気相手とLINE友達になってみた https://renta.papy.co.jp/renta/sc/frm/item/337767/title/2397031/


### Renta!独占
---


In [ ]:
import google_colab_selenium as gs
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
options.add_argument("--headless")

print("connectiong to remote browser...")
driver = gs.Chrome(options=options)

driver.get("https://renta.papy.co.jp/renta/sc/frm/page/original/c_comicspia.htm")

renta_original_data_to_insert = []

def get_renta_comicspia(number):
  topics_wrap_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]'
  )
  # topics_contents_innerwrapの個数をカウントする
  for topics_wrap in topics_wrap_list:
      inner_wrap_list = topics_wrap.find_elements(By.CLASS_NAME, 'topics_contents_innerwrap')
      count_inner_wrap_list = len(inner_wrap_list)
      # print("inner_wrap の個数:", len(inner_wrap_list))

  title_list = driver.find_elements(
      By.XPATH, '//*[@id="newContents"]/span'
  )
  for title in title_list:
    print("------")
    print("【" + title.text + "】")

  # topics_contents_innerwrapの中身取得
  for i in range(1,count_inner_wrap_list+1):
    book_title_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div[{i}]/div/div[1]/div/div[1]/span[2]'
  )
    for book in book_title_list:
      book_title = book.text

    book_link_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div[{i}]/div/div[1]/a'
  )
    for link in book_link_list:
      book_link = link.get_attribute("href")
    book_title = check_duplicate(book_title)
    book_title = delete_unnecessary_words(book_title)
    if book_title[:5] in [item[:5] for item in duplication_set] and book_title[-3:] in [item[-3:] for item in duplication_set]:
      continue
    print(book_title, book_link)
    renta_original_data_to_insert.append([book_title, book_link, "renta独占"])
    duplication_set.add(book_title)

def get_renta_comicspia2(number):
  # topics_wrap_list = driver.find_elements(
  #     By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]'
  # )
  # # topics_contents_innerwrapの個数をカウントする
  # for topics_wrap in topics_wrap_list:
  #     inner_wrap_list = topics_wrap.find_elements(By.CLASS_NAME, 'topics_contents_innerwrap')
  #     count_inner_wrap_list = len(inner_wrap_list)
  #     # print("inner_wrap の個数:", len(inner_wrap_list))

  # title_list = driver.find_elements(
  #     By.XPATH, '//*[@id="saleContents"]/span'
  # )
  # for title in title_list:
  #   print("------")
  #   print("【" + title.text + "】")

  # topics_contents_innerwrapの中身取得
  # for i in range(1,count_inner_wrap_list):
  #   book_title_list = driver.find_elements(
  #     By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div[{i}]/div/div[1]/div/div[2]'
  # )
  #   for book in book_title_list:
  #     book_title = book.text

  #   book_link_list = driver.find_elements(
  #     By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div[{i}]/div/div[1]/a'
  # )
  #   for link in book_link_list:
  #     book_link = link.get_attribute("href")
  #   book_title = check_duplicate(book_title)
  #   if book_title[:5] in [item[:5] for item in duplication_set] and book_title[-3:] in [item[-3:] for item in duplication_set]:
  #     continue
  #   print(book_title, book_link)
  #   renta_original_data_to_insert.append([book_title, book_link, "renta独占"])
  #   duplication_set.add(book_title)

  # 最下部取得
  title_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/h3/span[1]'
  )
  for title in title_list:
    print("------")
    print("【" + title.text + "】")

  # contents_list = driver.find_elements(
  #     By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div[4]/ul'
  # )
  contents_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div/ul'
  )

  # contents_listの個数をカウントする
  for content in contents_list:
      contents_li = content.find_elements(By.CLASS_NAME, 'contents_list')
      count_contents_li = len(contents_li)

  for li_content in range(1, count_contents_li+1):
    book_title_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div/ul/li[{li_content}]/a[1]/div[2]'
  )
    for book in book_title_list:
      book_title = book.text

    book_link_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div/ul/li[{li_content}]/a[1]'
  )
    for link in book_link_list:
      book_link = link.get_attribute("href")
    book_title = check_duplicate(book_title)
    book_title = delete_unnecessary_words(book_title)
    if book_title[:5] in [item[:5] for item in duplication_set] and book_title[-3:] in [item[-3:] for item in duplication_set]:
      continue
    print(book_title, book_link)
    renta_original_data_to_insert.append([book_title, book_link, "renta独占"])
    duplication_set.add(book_title)

def get_renta_comicspia3(number):
  topics_wrap_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]'
  )
  # topics_contents_innerwrapの個数をカウントする
  for topics_wrap in topics_wrap_list:
      inner_wrap_list = topics_wrap.find_elements(By.CLASS_NAME, 'topics_contents_innerwrap')
      count_inner_wrap_list = len(inner_wrap_list)
      # print("inner_wrap の個数:", len(inner_wrap_list))

  title_list = driver.find_elements(
      By.XPATH, '//*[@id="listContents"]/span'
  )
  for title in title_list:
    print("------")
    print("【" + title.text + "】")

  # topics_contents_innerwrapの中身取得
  for i in range(1,count_inner_wrap_list):
    book_title_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div[{i}]/div/div[1]/div/div[1]'
  )
    for book in book_title_list:
      book_title = book.text

    book_link_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div[{i}]/div/div[1]/a'
  )
    for link in book_link_list:
      book_link = link.get_attribute("href")
    book_title = check_duplicate(book_title)
    book_title = delete_unnecessary_words(book_title)
    if book_title[:5] in [item[:5] for item in duplication_set] and book_title[-3:] in [item[-3:] for item in duplication_set]:
      continue
    print(book_title, book_link)
    renta_original_data_to_insert.append([book_title, book_link, "renta独占"])
    duplication_set.add(book_title)

  # 最下部取得
  contents_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div[2]/ul'
  )

  # contents_listの個数をカウントする
  for content in contents_list:
      contents_li = content.find_elements(By.CLASS_NAME, 'contents_list')
      count_contents_li = len(contents_li)

  for li_content in range(1, count_contents_li+1):
    book_title_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div[2]/ul/li[{li_content}]/a[1]/div[2]'
  )
    for book in book_title_list:
      book_title = book.text

    book_link_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div[2]/ul/li[{li_content}]/a[1]'
  )
    for link in book_link_list:
      book_link = link.get_attribute("href")
    book_title = check_duplicate(book_title)
    book_title = delete_unnecessary_words(book_title)
    if book_title[:5] in [item[:5] for item in duplication_set] and book_title[-3:] in [item[-3:] for item in duplication_set]:
      continue
    print(book_title, book_link)
    renta_original_data_to_insert.append([book_title, book_link, "renta独占"])
    duplication_set.add(book_title)

def get_renta_comicspia4(number, title):
  title_list = driver.find_elements(
      By.XPATH, f'//*[@id="{title}"]/span'
  )
  for title in title_list:
    print("------")
    print("【" + title.text + "】")

  # 最下部取得
  contents_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div/ul'
  )

  # contents_listの個数をカウントする
  for content in contents_list:
      contents_li = content.find_elements(By.CLASS_NAME, 'contents_list')
      count_contents_li = len(contents_li)

  for li_content in range(1, count_contents_li+1):
    book_title_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div/ul/li[{li_content}]/a[1]/div[2]'
  )
    for book in book_title_list:
      book_title = book.text

    book_link_list = driver.find_elements(
      By.XPATH, f'//*[@id="comicspia"]/div[{number+1}]/div/ul/li[{li_content}]/a[1]'
  )
    for link in book_link_list:
      book_link = link.get_attribute("href")
    book_title = check_duplicate(book_title)
    book_title = delete_unnecessary_words(book_title)
    if book_title[:5] in [item[:5] for item in duplication_set] and book_title[-3:] in [item[-3:] for item in duplication_set]:
      continue
    print(book_title, book_link)
    renta_original_data_to_insert.append([book_title, book_link, "renta独占"])
    duplication_set.add(book_title)


get_renta_comicspia(1)
get_renta_comicspia2(2)
get_renta_comicspia3(3)
get_renta_comicspia4(4,"specialContents")
get_renta_comicspia4(5,"tatecomi")
get_renta_comicspia4(6,"enovel")

st.append_rows(values=renta_original_data_to_insert)

connectiong to remote browser...


<IPython.core.display.Javascript object>

------
【新刊をチェック！】
------
【期間限定で対象作品が無料！】
------
【COMICスピア作品一覧】
------
【特装版でまとめて読む！】
------
【アニコミも好評配信中！】
------
【オススメ特集！】
------
【タテコミ作品】
------
【絵ノベル版も要チェック！】


{'spreadsheetId': '1RiLchx4pkQ_vdngW3UIG49xuhoZEThM4rAhuTPJQdH4',
 'tableRange': "'シート65'!A1:C558",
 'updates': {'spreadsheetId': '1RiLchx4pkQ_vdngW3UIG49xuhoZEThM4rAhuTPJQdH4',
  'updatedRange': "'シート65'!A559"}}

### booklive オリジナル独占
---

In [ ]:
import requests
from bs4 import BeautifulSoup

get_dict = {"すべて": "", "少年・青年マンガ": "/comic", "少女・女性マンガ": "/comicf", "TL": "/teenslove", "BL": "/boyslove"}
booklive_data_to_insert = []

def get_booklive_comics(soup, title):
    roop_booklive(soup, 1, title)
    roop_booklive(soup, 2, title)

def roop_booklive(soup, number, title):
    for k in range(5):
        elems = soup.select(f'#area_new > div.area_inner > div:nth-of-type({number}) > div:nth-of-type({k+1}) > div.txtarea > div > a')
        if elems:
            link = elems[0]['href']
            full_path = "https://booklive.jp" + link

            res2 = requests.get(full_path)
            soup2 = BeautifulSoup(res2.text, 'html.parser')
            elems2 = soup2.select('#product_display_1')
            text = elems2[0].text.strip()
            text = check_duplicate(text)
            text = delete_unnecessary_words(text)
            if text[:5] in [item[:5] for item in duplication_set] and text[-3:] in [item[-3:] for item in duplication_set]:
              continue
            print(text, full_path)
            booklive_data_to_insert.append([text, full_path, "bookliveオリジナル独占"+ "【"+ title + "】"])
            duplication_set.add(text)

for k, v in get_dict.items():
    res = requests.get('https://booklive.jp/original-comic' + v)
    soup = BeautifulSoup(res.text, 'html.parser')
    kind_of_elems = soup.select('#area_new > div.heading_title.add_pickup_link > h2')
    print("------")
    print("【" + k + "】")
    get_booklive_comics(soup, k)

st.append_rows(values=booklive_data_to_insert)

------
【すべて】
------
【少年・青年マンガ】
------
【少女・女性マンガ】
------
【TL】
------
【BL】


{'spreadsheetId': '1RiLchx4pkQ_vdngW3UIG49xuhoZEThM4rAhuTPJQdH4',
 'tableRange': "'シート65'!A1:C558",
 'updates': {'spreadsheetId': '1RiLchx4pkQ_vdngW3UIG49xuhoZEThM4rAhuTPJQdH4',
  'updatedRange': "'シート65'!A559"}}

### まんが王国 ランキング
---

In [ ]:
import google_colab_selenium as gs
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
options.add_argument("--headless")

print("connectiong to remote browser...")
driver = gs.Chrome(options=options)

driver.get("https://comic.k-manga.jp/rank/")

kingmanga_data_to_insert = []
ranking_item_wrap_list = driver.find_elements(
    By.XPATH,
    '//*[@id="contents"]/section[2]/ul/li'
)

for ranking_item_wrap in ranking_item_wrap_list:
  count = 0
  manga_title = ranking_item_wrap.find_element(
      By.XPATH,
      "./a/h2",
  ).text

  manga_url = ranking_item_wrap.find_element(
      By.XPATH,
      "./a",
  ).get_attribute("href")
  manga_title = check_duplicate(manga_title)
  text = delete_unnecessary_words(text)
  if manga_title[:5] in [item[:5] for item in duplication_set] and manga_title[-3:] in [item[-3:] for item in duplication_set]:
    continue
  print(f"{manga_title} {manga_url}")
  kingmanga_data_to_insert.append([manga_title, manga_url, "まんが王国ランキング"])
  duplication_set.add(manga_title)

  count += 1
  if count == 20:
      break
st.append_rows(values=kingmanga_data_to_insert)
driver.quit()

connectiong to remote browser...


<IPython.core.display.Javascript object>

### まんが王国 先行配信ランキング
---

In [ ]:
import google_colab_selenium as gs
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
options.add_argument("--headless")

print("connectiong to remote browser...")
driver = gs.Chrome(options=options)

driver.get("https://comic.k-manga.jp/pre")
kingmanga_ranking_data_to_insert = []

ranking_item_wrap_list = driver.find_elements(
    By.XPATH,
    '//*[@id="contents"]/section[2]/div[2]/div'
)

for ranking_item_wrap in ranking_item_wrap_list:

  manga_title = ranking_item_wrap.find_element(
      By.XPATH,
      "./a/p[2]",
  ).text

  manga_url = ranking_item_wrap.find_element(
      By.XPATH,
      "./a",
  ).get_attribute("href")
  manga_title = check_duplicate(manga_title)
  manga_title = delete_unnecessary_words(manga_title)
  if manga_title[:5] in [item[:5] for item in duplication_set] and manga_title[-3:] in [item[-3:] for item in duplication_set]:
    continue
  print(f"{manga_title} {manga_url}")
  kingmanga_ranking_data_to_insert.append([manga_title, manga_url, "まんが王国先行配信ランキング"])
  duplication_set.add(manga_title)

st.append_rows(values=kingmanga_ranking_data_to_insert)
driver.quit()

connectiong to remote browser...


<IPython.core.display.Javascript object>

### めちゃコミックス 独占
---


In [ ]:
import requests
from bs4 import BeautifulSoup

# スクレイピング対象の URL にリクエストを送り HTML を取得する
res = requests.get('https://mechacomic.jp/books/exclusive_top')

mecha_comi_list = []
  # レスポンスの HTML から BeautifulSoup オブジェクトを作る
soup = BeautifulSoup(res.text, 'html.parser')

title = soup.select(f'body > div.pc.all > div > div.container_body.right.u-mb24 > section > section:nth-child(4) > h2')
print("------")
print("【"+ title[0].text.strip() + "】")

elems = soup.select(f'#ec_exclusive_ad_slider_1 > span')

# liの個数をカウントする
target_element = soup.select('body > div.pc.all > div > div.container_body.right.u-mb24 > section > section:nth-of-type(4) > div > ul')
new = soup.find(class_='p-exclusiveImage').find("ul")
li_count = 0
for element in new.find_all("li"):
  li_count+=1

for i in range(1, li_count * 2, 2):
  elems = soup.select(f'#ec_exclusive_ad_slider_{i}')
  title_elems = soup.select(f'#ec_exclusive_ad_slider_{i} > span')
  if elems:
    book_title = title_elems[0].text.strip()
    link = elems[0]['href']
    full_path = "https://mechacomic.jp" + link
    book_title = check_duplicate(book_title)
    book_title = delete_unnecessary_words(book_title)
    if book_title[:5] in [item[:5] for item in duplication_set] and book_title[-3:] in [item[-3:] for item in duplication_set]:
      continue
    print(book_title, full_path)
    duplication_set.add(book_title)
    mecha_comi_list.append([book_title, full_path, "めちゃコミ独占"])


mechacomic_dict = {"おすすめ無料連載" : "#ec_exclusive_top_free_daily_", "おすすめ期間限定無料" : "#ec_exclusive_top_free_discount_", "あなたにおすすめ" : "#ec_exclusive_top_recli_", "人気作品ランキング(少女・女性)" : "#ec_exclusive_top_ranking_girl_", "人気作品ランキング(少年・青年)" : "#ec_exclusive_top_ranking_boy_", "新着入荷" : "#ec_exclusive_top_recent_", "おすすめレーベル" : "#ec_exclusive_fufu_"}

def get_mechacomic_books(count):
  title_element = soup.select(f'body > div.pc.all > div > div.container_body.right.u-mb24 > section > section:nth-of-type({count}) > a > h2')
  title = title_element[0].text.strip()
  mechacomic_class = mechacomic_dict[title]
  print("------")
  print("【"+ title + "】")

  target_element = soup.select(f'body > div.pc.all > div > div.container_body.right.u-mb24 > section > section:nth-of-type({count}) > ul')
  ul_element = target_element[0] if target_element else None
  if ul_element:
      li_elements = ul_element.find_all('li')
      li_count = len(li_elements)

  for i in range(1, li_count * 2, 2):
    elems = soup.select(f'{mechacomic_class}{i}')
    title_elems = soup.select(f'{mechacomic_class}{i} > span')
    if elems:
      book_title = title_elems[0].text.strip()
      link = elems[0]['href']
      full_path = "https://mechacomic.jp" + link
      book_title = check_duplicate(book_title)
      book_title = delete_unnecessary_words(book_title)
      if book_title[:5] in [item[:5] for item in duplication_set] and book_title[-3:] in [item[-3:] for item in duplication_set]:
        continue
      print(book_title, full_path)
      mecha_comi_list.append([book_title, full_path, "めちゃコミ独占"])
      duplication_set.add(book_title)

def get_mechacomic_books2(count):
  title_element = soup.select(f'body > div.pc.all > div > div.container_body.right.u-mb24 > section > section:nth-of-type({count}) > h2')
  title = title_element[0].text.strip()
  mechacomic_class = mechacomic_dict[title]
  print("------")
  print("【"+ title + "】")

  target_element = soup.select(f'body > div.pc.all > div > div.container_body.right.u-mb24 > section > section:nth-of-type({count}) > div > ul')

  ul_element = target_element[0] if target_element else None
  if ul_element:
      li_elements = ul_element.find_all('li')
      li_count = len(li_elements)

  for i in range(1, li_count * 2, 2):
    elems = soup.select(f'{mechacomic_class}{i}')
    title_elems = soup.select(f'{mechacomic_class}{i} > span')
    if elems:
      book_title = title_elems[0].text.strip()
      link = elems[0]['href']
      full_path = "https://mechacomic.jp" + link
      book_title = check_duplicate(book_title)
      book_title = delete_unnecessary_words(book_title)
      if book_title[:5] in [item[:5] for item in duplication_set] and book_title[-3:] in [item[-3:] for item in duplication_set]:
        continue
      print(book_title, full_path)
      mecha_comi_list.append([book_title, full_path, "めちゃコミ独占"])
      duplication_set.add(book_title)

def get_mechacomic_books3(count):
  title_element = soup.select(f'body > div.pc.all > div > div.container_body.right.u-mb24 > section > section:nth-of-type(7) > a > h2')
  title = title_element[0].text.strip()
  mechacomic_class = mechacomic_dict[title]
  print("------")
  print("【"+ title + "】")

  target_element = soup.select(f'body > div.pc.all > div > div.container_body.right.u-mb24 > section > section:nth-of-type({count}) > div > ul')

  ul_element = target_element[0] if target_element else None
  if ul_element:
      li_elements = ul_element.find_all('li')
      li_count = len(li_elements)

  for i in range(1, li_count * 2, 2):
    elems = soup.select(f'{mechacomic_class}{i}')
    title_elems = soup.select(f'{mechacomic_class}{i} > span')
    if elems:
      book_title = title_elems[0].text.strip()
      link = elems[0]['href']
      full_path = "https://mechacomic.jp" + link
      book_title = check_duplicate(book_title)
      book_title = delete_unnecessary_words(book_title)
      if book_title[:5] in [item[:5] for item in duplication_set] and book_title[-3:] in [item[-3:] for item in duplication_set]:
        continue
      print(book_title, full_path)
      mecha_comi_list.append([book_title, full_path, "めちゃコミ独占"])
      duplication_set.add(book_title)

def get_mechacomic_books4(count):
  title_element = soup.select(f'body > div.pc.all > div > div.container_body.right.u-mb24 > section > section:nth-of-type({count}) > h2')
  title = title_element[0].text.strip()
  mechacomic_class = mechacomic_dict[title]
  print("------")
  print("【"+ title + "】")

  target_element = soup.select(f'body > div.pc.all > div > div.container_body.right.u-mb24 > section > section:nth-of-type(8) > section:nth-of-type(2) > div > ul')

  ul_element = target_element[0] if target_element else None
  if ul_element:
      li_elements = ul_element.find_all('li')
      li_count = len(li_elements)

  for i in range(1, li_count * 2, 2):
    elems = soup.select(f'{mechacomic_class}{i}')
    title_elems = soup.select(f'{mechacomic_class}{i} > span')
    if elems:
      book_title = title_elems[0].text.strip()
      link = elems[0]['href']
      full_path = "https://mechacomic.jp" + link
      book_title = check_duplicate(book_title)
      book_title = delete_unnecessary_words(book_title)
      if book_title[:5] in [item[:5] for item in duplication_set] and book_title[-3:] in [item[-3:] for item in duplication_set]:
        continue
      print(book_title, full_path)
      mecha_comi_list.append([book_title, full_path, "めちゃコミ独占"+ "【"+ title + "】"])
      duplication_set.add(book_title)

get_mechacomic_books(2)
get_mechacomic_books(3)
get_mechacomic_books2(5)
get_mechacomic_books2(6)
get_mechacomic_books3(7)
get_mechacomic_books4(8)
st.append_rows(values=mecha_comi_list)


------
【いま話題の作品】
------
【おすすめ無料連載】
------
【おすすめ期間限定無料】
------
【人気作品ランキング(少女・女性)】
------
【人気作品ランキング(少年・青年)】
------
【新着入荷】
------
【おすすめレーベル】


{'spreadsheetId': '1RiLchx4pkQ_vdngW3UIG49xuhoZEThM4rAhuTPJQdH4',
 'tableRange': "'シート65'!A1:C558",
 'updates': {'spreadsheetId': '1RiLchx4pkQ_vdngW3UIG49xuhoZEThM4rAhuTPJQdH4',
  'updatedRange': "'シート65'!A559"}}

### ピッコマ ランキング
---

In [ ]:
import requests
from bs4 import BeautifulSoup

piccoma_list = []
classification_dict = {"総合": "0","ファンタジー": "2","恋愛": "1","アクション": "5","ドラマ": "3","ホラー・ミステリー": "7","裏社会・アングラ": "9","スポーツ": "6","グルメ": "10","日常": "4","TL": "13","BL": "14",}


def get_piccoma_books(classification):
  page_query = classification_dict[classification]
  # スクレイピング対象の URL にリクエストを送り HTML を取得する
  res = requests.get(f'https://piccoma.com/web/ranking/K/P/{page_query}')
  # レスポンスの HTML から BeautifulSoup オブジェクトを作る
  soup = BeautifulSoup(res.text, 'html.parser')
  print("------")
  print("【"+ classification + "】")
  for i in range(1,11):
    title_elems = soup.select(f'#js_contentBody > section > ul > li:nth-of-type({i}) > a > div > div.PCM-l_rankingProduct_info > div > div.PCM-rankingProduct_tdata > div.PCM-rankingProduct_title > p')
    link_elems = soup.select(f'#js_contentBody > section > ul > li:nth-child({i}) > a')
    link = link_elems[0]['href']
    full_path = "https://piccoma.com/" + link
    title = title_elems[0].text.strip()
    title = check_duplicate(title)
    title = delete_unnecessary_words(title)
    if title[:5] in [item[:5] for item in duplication_set] and title[-3:] in [item[-3:] for item in duplication_set]:
      continue
    print(title, full_path)
    piccoma_list.append([title, full_path, "ピッコマランキング"+ "【"+ classification + "】"])
    duplication_set.add(title)

get_piccoma_books("総合")
get_piccoma_books("ファンタジー")
get_piccoma_books("恋愛")
get_piccoma_books("アクション")
get_piccoma_books("ドラマ")
get_piccoma_books("ホラー・ミステリー")
get_piccoma_books("裏社会・アングラ")
get_piccoma_books("スポーツ")
get_piccoma_books("グルメ")
get_piccoma_books("日常")
get_piccoma_books("TL")
get_piccoma_books("BL")

st.append_rows(values=piccoma_list)

------
【総合】
------
【ファンタジー】
------
【恋愛】
------
【アクション】
------
【ドラマ】
------
【ホラー・ミステリー】
------
【裏社会・アングラ】
------
【スポーツ】
------
【グルメ】
------
【日常】
------
【TL】
------
【BL】


{'spreadsheetId': '1RiLchx4pkQ_vdngW3UIG49xuhoZEThM4rAhuTPJQdH4',
 'tableRange': "'シート65'!A1:C558",
 'updates': {'spreadsheetId': '1RiLchx4pkQ_vdngW3UIG49xuhoZEThM4rAhuTPJQdH4',
  'updatedRange': "'シート65'!A559"}}

ここまでの手順が完了したら、kw・発注管理シートの"スクレイピング結果"シートを確認し、漫画やurlが記載されていることを確認してください。

### 発注管理シートの電子書籍C列と比較をし、被りがなければシートに追加する
---

全てのセル実行が完了してから動かしてください。

#### 事前準備
Google Sheets APIへの書き込みリクエストの制限を超えるため、リスト系は作成しておく

In [ ]:
# スプレッドシートとの連携
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# 発注管理シート
url = "https://docs.google.com/spreadsheets/d/1RiLchx4pkQ_vdngW3UIG49xuhoZEThM4rAhuTPJQdH4/edit#gid=1338119606"
ss = gc.open_by_url(url)
# シートを特定する（書き出し用）
st = ss.worksheet("スクレイピング結果")

# シートを特定する（電子書籍）
el_st = ss.worksheet("電子書籍")

In [ ]:
# C列の値を取得してリストに格納
c_column_values = el_st.col_values(3)
# A列とB列の値を取得
a_values = st.col_values(1)[1:]
b_values = st.col_values(2)[1:]
c_values = st.col_values(3)[1:]

# 対応する辞書を作成
corresponding_dict = dict(zip(a_values, b_values))
corresponding_dict2 = dict(zip(a_values, c_values))

last_row = len(el_st.col_values(3)) + 1

#### 被りチェック -> 発注管理シート書き込み

In [ ]:
# 一括で更新するデータを作成
service_dict = {"https://www.cmoa.jp":"コミックシーモア", "https://booklive.jp":"ブックライブ", "https://renta.papy.co.jp":"renta", "https://comic.k-manga.jp":"まんが王国", "https://mechacomic.jp":"めちゃコミック", "https://piccoma.com" : "ピッコマ"}

batch_data = []
roop_counter = 0
# 最初の5文字が一緒 かつ 後ろの3文字がc列のキーワードに含まれる
for key, value in corresponding_dict.items():
    # c_column_valuesリストの各要素の最初の5文字と比較
    if any(key[:5] == c_prefix[:5] and key[-3:] in c_prefix for c_prefix in c_column_values):
        print(key)
        continue
    if key not in c_column_values:
        insert_row = last_row + roop_counter
        batch_data.append({"range": f"C{insert_row}", "values": [[key]]})
        batch_data.append({"range": f"N{insert_row}", "values": [["新規"]]})
        batch_data.append({"range": f"T{insert_row}", "values": [[value]]})
        url_prefix = next((service_url for service_url in service_dict.keys() if value.startswith(service_url)), None)
        if url_prefix:
            # service_name = service_dict[url_prefix]
            memo = corresponding_dict2[key]
            # el_st.update_cell(insert_row, 10, service_name + memo)
            batch_data.append({"range": f"J{insert_row}", "values": [[memo]]})
        roop_counter += 1

# 一括で更新
el_st.batch_update(batch_data)

# 全てのkw洗い出しが完了したら、スクレイピング結果のシートを白紙に戻す
delete_row = len(st.col_values(1))
print(delete_row)
# 行を削除する
st.delete_rows(2, delete_row + 1)

赤ちゃんに転生した話
